In [12]:
pip install psycopg2-binary pandas openpyxl sqlalchemy redshift_connector


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 484.1 kB/s eta 0:00:00a 0:00:01
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.7 MB/s eta 0:00:00
  Using cached charset_normalizer-3.3.2-cp39-cp39-macosx_11_0_arm64.whl.metadata (33 kB)
  Using cached idna-3.7-py3-none-any.whl.metadata (9.9 kB)
  Using cached certifi-2024.7.4-py3-none-any.whl.metadata (2.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.0/130.0 kB 1.2 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 28.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 37.1 MB/s eta 0:00:0000:0100:01
Using cached requests-2.32.3-py3-none-any.whl (

In [21]:
import redshift_connector
import pandas as pd
from openpyxl import load_workbook, Workbook
from getpass import getpass
import os

# Prompt for user credentials
user = input("Enter your username: ")
password = getpass("Enter your password: ")

# Database connection details for Redshift
db_config = {
    'dbname': 'analytics',
    'user': user,
    'password': password,
    'host': '127.0.0.1',
    'port': '17592'  # Default port for Redshift
}

# Connect to Redshift using redshift_connector
conn = redshift_connector.connect(
    host=db_config['host'],
    port=int(db_config['port']),
    database=db_config['dbname'],
    user=db_config['user'],
    password=db_config['password']
)

# List of SQL queries and their corresponding sheet names
queries = {

'NEW_KYC': """
        --NEW KYC
--KYC PASSED USER
WITH base as (
SELECT
last_day( TO_CHAR(DATEADD('months', -number, date_trunc('days',GETDATE())), 'yyyy-mm-dd')::date) AS created_at
FROM ref_numbers
WHERE id <= DATE_DIFF('months', '2022-12-01', GETDATE()) -- set start date
ORDER BY 1 ASC
)
, v_users as (
select *
, case when user_type is null then 'RLU' else user_type end as user_type_adjusted
from workspace.sam_RLU_NU_v1
)
, nokyc_mtd_before_jul AS (
SELECT
last_day(registration_date_est5edt)::DATE AS date
, user_type_adjusted
, count(1) AS passed_mtd
FROM v_users
WHERE manual_kyc_passed_date_est5edt IS NULL
AND org_name = 'Sesame Cash'
AND registration_date_est5edt >= '2023/01/01 00:00'
AND status = 'REGISTERED'
and date_part('d',registration_date_est5edt) <= EXTRACT(day FROM CURRENT_DATE - INTERVAL '3 day')--change the number of days here 
and registration_date_est5edt::date < '2023-07-01'
GROUP BY 1,2
)
, nokyc_mo_before_jul AS (
SELECT
last_day(registration_date_est5edt)::DATE AS date
, user_type_adjusted
, count(1) AS passed_mo
FROM v_users
WHERE manual_kyc_passed_date_est5edt IS NULL
AND org_name = 'Sesame Cash'
AND registration_date_est5edt >= '2023/01/01 00:00'
AND status = 'REGISTERED'
and registration_date_est5edt::date < '2023-07-01'
GROUP BY 1,2
)
, prod as (
select e.usertoken,
effective_start_datetime,
enrollment_status,
enrollment_timestamp,
convert_timezone('America/Los_Angeles', enrollment_timestamp) as enrollment_timestamp_PDT,
last_day(trunc(convert_timezone('America/Los_Angeles', acct_registration_complete_datetime)))as registration_date,
case when registration_date=last_day(effective_start_datetime) then 'NU' else 'RLU' end as user_type
from prod.sesame_cash.sesame_cash_enrollment e
join prod.public.user u
on e.usertoken = u.usertoken
where effective_end_datetime='2100-01-01 00:00:00'
and enrollment_timestamp is not null
and enrollment_timestamp::date >='2023-01-01'
)
, rlu as (
select *
from
stack.v_users a
left join prod b
on a.partner_user_id=b.usertoken
where created_at_est5edt>='2023-01-01'
and org_name='Sesame Cash'
)
, v_users_new as (
select *
, case when user_type is null then 'RLU' else user_type end as user_type_adjusted
from rlu
)
, nokyc_mtd AS (
SELECT
last_day(registration_date_est5edt)::DATE AS date
, user_type_adjusted
, count(1) AS passed_mtd
FROM v_users_new
WHERE manual_kyc_passed_date_est5edt IS NULL
AND org_name = 'Sesame Cash'
AND registration_date_est5edt >= '2023/01/01 00:00'
AND status = 'REGISTERED'
and date_part('d',registration_date_est5edt) <= EXTRACT(day FROM CURRENT_DATE - INTERVAL '3 day') --change the number of days here 
and registration_date_est5edt::date >= '2023-07-01'
GROUP BY 1,2
)
, nokyc AS (
SELECT
last_day(registration_date_est5edt)::DATE AS date
, user_type_adjusted
, count(1) AS passed
FROM v_users_new
WHERE manual_kyc_passed_date_est5edt IS NULL
AND org_name = 'Sesame Cash'
AND registration_date_est5edt >= '2023/01/01 00:00'
and registration_date_est5edt::date >= '2023-07-01'
AND status = 'REGISTERED'
GROUP BY 1,2
)
, nokyc_mtd_final AS(
SELECT *
FROM nokyc_mtd_before_jul
UNION 
SELECT *
FROM nokyc_mtd
)
, nokyc_final AS(
SELECT *
FROM nokyc_mo_before_jul
UNION 
SELECT *
FROM nokyc
)
SELECT
date_trunc('mon',created_at)::date AS mo
, e.user_type_adjusted
, passed_mtd
, passed_mo
FROM base a
LEFT JOIN nokyc_final e ON a.created_at = e.date 
LEFT JOIN nokyc_mtd_final m ON a.created_at = m.date 
        AND e.user_type_adjusted =  m.user_type_adjusted
ORDER BY 1,2;
    """,


'KYC_Passed_After_FAILING': """
       --KYC PASSED AFTER FAILING USER + MTD
with prod as (
select e.usertoken,
effective_start_datetime,
enrollment_status,
enrollment_timestamp,
convert_timezone('America/Los_Angeles', enrollment_timestamp) as enrollment_timestamp_PDT,
last_day(trunc(convert_timezone('America/Los_Angeles', acct_registration_complete_datetime)))as registration_date,
case when registration_date=last_day(effective_start_datetime) then 'NU' else 'RLU' end as user_type
from prod.sesame_cash.sesame_cash_enrollment e
join prod.public.user u
on e.usertoken = u.usertoken
where effective_end_datetime='2100-01-01 00:00:00'
and enrollment_timestamp is not null
and enrollment_timestamp::date >='2023-01-01'
)
, rlu as (
select *
from
stack.v_users a
left join prod b
on a.partner_user_id=b.usertoken
where created_at_est5edt>='2023-01-01'
and org_name='Sesame Cash'
)
, base as (
SELECT
last_day( TO_CHAR(DATEADD('months', -number, date_trunc('days',GETDATE())), 'yyyy-mm-dd')::date) AS created_at
FROM ref_numbers
WHERE id <= DATE_DIFF('months', '2022-12-01', GETDATE()) -- set start date
ORDER BY 1 ASC
)
, v_users as (
select *
, case when user_type is null then 'RLU' else user_type end as user_type_adjusted
from rlu
)
, enrolled_mtd AS (
SELECT DISTINCT
last_day(registration_date_est5edt)::DATE AS date
, user_type_adjusted
, count(1) AS passed_after_failing_mtd
FROM v_users
WHERE manual_kyc_passed_date_est5edt IS NOT NULL
AND org_name = 'Sesame Cash'
AND registration_date_est5edt >= '2023/01/01 00:00'
AND status = 'REGISTERED'
and date_part('d',registration_date_est5edt) <= EXTRACT(day FROM CURRENT_DATE - INTERVAL '3 day') --change the days here
GROUP BY 1,2
ORDER BY 1
)
, enrolled AS (
SELECT DISTINCT
last_day(registration_date_est5edt)::DATE AS date
, user_type_adjusted
, count(1) AS passed_after_failing
FROM v_users
WHERE manual_kyc_passed_date_est5edt IS NOT NULL
AND org_name = 'Sesame Cash'
AND registration_date_est5edt >= '2023/01/01 00:00'
AND status = 'REGISTERED'
GROUP BY 1,2
ORDER BY 1
)
SELECT
date_trunc('mon',created_at)::date AS mo
, e.user_type_adjusted
, passed_after_failing_mtd
, passed_after_failing
FROM base a
LEFT JOIN enrolled e ON a.created_at = e.date 
LEFT JOIN enrolled_mtd m ON a.created_at = m.date 
        AND e.user_type_adjusted =  m.user_type_adjusted
ORDER BY 1,2;
""",
    
    




'Failed_KYC': """ --FAILED KYC
WITH base as (
SELECT
last_day( TO_CHAR(DATEADD('months', -number, date_trunc('days',GETDATE())), 'yyyy-mm-dd')::date) AS created_at
FROM ref_numbers
WHERE id <= DATE_DIFF('months', '2022-12-01', GETDATE()) -- set start date
ORDER BY 1 ASC
)
, v_users as (
select *
, case when user_type is null then 'RLU' else user_type end as user_type_adjusted
from workspace.sam_RLU_NU_v1
)
, prod as (
select e.usertoken,
effective_start_datetime,
enrollment_status,
enrollment_timestamp,
convert_timezone('America/Los_Angeles', enrollment_timestamp) as enrollment_timestamp_PDT,
last_day(trunc(convert_timezone('America/Los_Angeles', acct_registration_complete_datetime)))as registration_date,
case when registration_date=last_day(effective_start_datetime) then 'NU' else 'RLU' end as user_type
from prod.sesame_cash.sesame_cash_enrollment e
join prod.public.user u
on e.usertoken = u.usertoken
where effective_end_datetime='2100-01-01 00:00:00'
and enrollment_timestamp is not null
and enrollment_timestamp::date >='2023-01-01'
)
, rlu as (
select *
from stack.v_users a
left join prod b
on a.partner_user_id=b.usertoken
where created_at_est5edt>='2023-01-01'
and org_name='Sesame Cash'
)
,failed_kyc_before_mtd AS(
SELECT
last_day(created_at_est5edt)::DATE AS date
, user_type_adjusted
, count(1) AS failed_kyc_mtd
FROM v_users
WHERE org_name = 'Sesame Cash'
AND manual_kyc_passed_date_est5edt IS NULL
AND created_at_est5edt::date >= '2023-01-01'
AND status = 'FAILED_KYC'
AND date_part('d',created_at_est5edt) <= EXTRACT(day FROM CURRENT_DATE - INTERVAL '3 day') -- change the number of days here
AND created_at_est5edt::date < '2023-08-01'
GROUP BY 1,2
ORDER BY 1
)
,failed_kyc_before_mo AS(
SELECT
last_day(created_at_est5edt)::DATE AS date
, user_type_adjusted
, count(1) AS failed_kyc_mo
FROM v_users
WHERE org_name = 'Sesame Cash'
AND manual_kyc_passed_date_est5edt IS NULL
AND created_at_est5edt::date >= '2023-01-01'
AND status = 'FAILED_KYC'
AND created_at_est5edt::date < '2023-08-01'
GROUP BY 1,2
ORDER BY 1
)
, v_users_new as (
select *
, case when user_type is null then 'RLU' else user_type end as user_type_adjusted
from rlu
)
,failed_kyc_mtd AS(
SELECT
last_day(created_at_est5edt)::DATE AS date
, user_type_adjusted
, count(1) AS failed_kyc_mtd
FROM v_users_new
WHERE org_name = 'Sesame Cash'
AND manual_kyc_passed_date_est5edt IS NULL
AND created_at_est5edt::date >= '2023-08-01'
AND status = 'FAILED_KYC'
and date_part('d',created_at_est5edt) <= EXTRACT(day FROM CURRENT_DATE - INTERVAL '3 day') --change the days here
GROUP BY 1,2
ORDER BY 1
),failed_kyc AS(
SELECT
last_day(created_at_est5edt)::DATE AS date
, user_type_adjusted
, count(1) AS failed_kyc
FROM v_users_new
WHERE org_name = 'Sesame Cash'
AND manual_kyc_passed_date_est5edt IS NULL
AND created_at_est5edt::date >= '2023-08-01'
AND status = 'FAILED_KYC'
GROUP BY 1,2
ORDER BY 1
), failed_kyc_mtd_final AS(
SELECT *
FROM failed_kyc_before_mtd
UNION
SELECT *
FROM failed_kyc_mtd

), failed_kyc_final AS(
SELECT *
FROM failed_kyc_before_mo
UNION 
SELECT *
FROM failed_kyc
)
SELECT
date_trunc('mon',created_at)::date AS mo
, e.user_type_adjusted
, failed_kyc_mtd
, failed_kyc_mo
FROM base a
LEFT JOIN failed_kyc_final e ON a.created_at = e.date 
LEFT JOIN failed_kyc_mtd_final m ON a.created_at = m.date 
        AND e.user_type_adjusted =  m.user_type_adjusted
ORDER BY 1,2;

""",

   'NEW_CB': """
        --NEW CB
with base as (
select 
DATEADD('months', -number, date_trunc('month',GETDATE()))::date AS created_at
from ref_numbers
where id <= DATE_DIFF('months', '2022-12-01', GETDATE()) -- set start date
order by 1 
), cb2_base as (
select
i2c_customer_id
, opt_in_date::date as cb2_opt_date
, status
, created_at_est5edt::date as creation_date
, registration_date_est5edt::date as registration_date
, received_first_fund_date::date as funding_date
, first_funding_date::date as money_hit_account_date
, w.usertoken as pilot_user
, case when registration_date<'2023-08-15' and w.usertoken is null then 'existing Cash Users'
else 'GA enrolled Users' end as cash_status
from analytics.stack.secured_card_settings cb
left join analytics.stack.v_users a
on a.user_id=cb.user_id
left join ssrp_us.v_user_cohort_first_funding f
on f.customer_id=a.i2c_customer_id
left join workspace.sam_CB2_whitelist_all_cohorts_july14 w
on w.usertoken=a.partner_user_id
where cb_version= '2'
and is_opted_in='TRUE'
), cb2_funded_monthly as (
select date_trunc('mon',local_date_time)::date 
, count(distinct a.customer_id) as cb2_funded
from stack.v_event_logs_i2c_alert_incoming a
inner join cb2_base b
on a.customer_id=b.i2c_customer_id
left join ssrp_us.v_user_cohort_first_funding f
on a.customer_id = f.customer_id
where local_date_time::date >= '2023-07-01'
and card_program_id='CS Secured M CB2'
and first_funding_month >= '2023-08-31'
and service='Activate Card'
group by 1
), cb2_funded_mtd as (
select date_trunc('mon',local_date_time)::date 
, count(distinct a.customer_id) as cb2_funded_mtd
from stack.v_event_logs_i2c_alert_incoming a
inner join cb2_base b
on a.customer_id=b.i2c_customer_id
left join ssrp_us.v_user_cohort_first_funding f
on a.customer_id = f.customer_id
where local_date_time::date >= '2023-07-01'
and card_program_id='CS Secured M CB2'
and first_funding_month >= '2023-08-31'
and date_part('d',local_date_time) <= EXTRACT(day FROM CURRENT_DATE - INTERVAL '3 day') -- change the number of days here
and service='Activate Card'
group by 1
), mtd as(
select date_trunc('mon',first_credit_builder_month)::date
, sum(case when b.customer_id is not null then 1 else 0 end) as new_funded_cb_users_mtd
from ssrp_us.v_user_cohort_credit_builder a
left join ssrp_us.v_user_cohort_first_funding b
on a.customer_id=b.customer_id
and a.first_credit_builder_month=b.first_funding_month
where date_part('d', first_credit_builder_date) <= EXTRACT(day FROM CURRENT_DATE - INTERVAL '3 day')-- change the number of days here
and first_credit_builder_month >= '2023-01-01'
group by 1
order by 1
), monthly as (
select date_trunc('mon',first_credit_builder_month)::date
, sum(case when b.customer_id is not null then 1 else 0 end) as new_funded_cb_users
from ssrp_us.v_user_cohort_credit_builder a
left join ssrp_us.v_user_cohort_first_funding b
on a.customer_id=b.customer_id
and a.first_credit_builder_month=b.first_funding_month
where first_credit_builder_month >= '2023-01-01'
group by 1
order by 1
), cb_enroll as (
select
 date_trunc('mon',opt_in_date::date) as opt_in_mo
, SUM(CASE WHEN cb_version='2' and received_first_fund='TRUE' THEN 1 END) AS cb2_funded_enrolled
, SUM(CASE WHEN cb_version='2' and received_first_fund='TRUE' and date_part ('d',opt_in_date::date)<= EXTRACT(day FROM CURRENT_DATE - INTERVAL '3 day') THEN 1 END) AS cb2_funded_enrolled_mtd -- change the number of days here
from stack.v_users a
left join analytics.stack.secured_card_settings cb
on a.user_id=cb.user_id
left join ssrp_us.v_user_cohort_first_funding f
on f.customer_id=a.i2c_customer_id
where  org_name='Sesame Cash'
and is_opted_in='TRUE'
group by 1
order by 1
)
select b.created_at
,nvl(mtd.new_funded_cb_users_mtd,0) + nvl(cb2_funded_mtd,0) as new_funded_cb_mtd
,cb2_funded_enrolled_mtd
,nvl(m.new_funded_cb_users,0) + nvl(cb2_funded,0) as new_funded_cb_monthly
,cb2_funded_enrolled
from base b
left join monthly m on b.created_at = m.date_trunc
left join mtd on b.created_at = mtd.date_trunc
left join cb2_funded_monthly on b.created_at = cb2_funded_monthly.date_trunc
left join cb2_funded_mtd on b.created_at = cb2_funded_mtd.date_trunc
left join cb_enroll on b.created_at = cb_enroll.opt_in_mo
order by 1
;

    """,

}

# Fetch data using the provided query
def fetch_data(query):
    with conn.cursor() as cursor:
        cursor.execute(query)
        result = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
    df = pd.DataFrame(result, columns=columns)
    return df

excel_file_path = 'Test.xlsx'

if not os.path.exists(excel_file_path):
    # Create a new Excel file if it does not exist
    wb = Workbook()
    wb.save(excel_file_path)

# Load the existing Excel file
try:
    book = load_workbook(excel_file_path)
except Exception as e:
    print(f"Error loading workbook: {e}")
    book = Workbook()
    book.save(excel_file_path)
    book = load_workbook(excel_file_path)

# Iterate over the queries and save each result to a separate sheet
with pd.ExcelWriter(excel_file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    for sheet_name, query in queries.items():
        # Fetch data from the database
        data = fetch_data(query)
        
        # Remove the existing sheet if it exists
        if sheet_name in book.sheetnames:
            del book[sheet_name]
        
        # Write the DataFrame to the Excel file
        data.to_excel(writer, sheet_name=sheet_name, index=False)

print("Data successfully written to Excel file.")

# Close the Redshift connection
conn.close()


Data successfully written to Excel file.
